In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from pathlib import Path
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
import holidays
from sklearn.impute import SimpleImputer
import utils

In [2]:
data = pd.read_parquet(Path("data") / "train.parquet")

In [3]:
print(type(data))

<class 'pandas.core.frame.DataFrame'>


# Preprocess weather data

In [4]:
weather_data = pd.read_csv(Path("data") / "external_data.csv", parse_dates=["date"])

In [5]:
# Columns to keep from EDA analysis
columns_to_keep = ["date", "ff", "t", "ht_neige", "rr1", "ww", "n", "etat_sol"]

weather_data = weather_data[columns_to_keep]

# Display the first few rows of the filtered dataset
display(weather_data.head())

,date,ff,t,ht_neige,rr1,ww,n,etat_sol
0,2021-01-01 00:00:00,1.8,272.75,0.00,0.0,2,10.0,1.0
1,2021-01-01 03:00:00,1.7,271.25,0.00,0.0,40,25.0,1.0
2,2021-01-01 06:00:00,2.6,271.95,0.00,0.0,3,90.0,1.0
3,2021-01-01 09:00:00,1.7,272.45,0.01,0.0,10,50.0,13.0
4,2021-01-01 12:00:00,1.0,276.95,-0.01,0.0,2,90.0,11.0


In [6]:
# Check for duplicates 
duplicate_rows = weather_data.duplicated()

num_duplicates = duplicate_rows.sum()
print(f"Number of duplicate rows: {num_duplicates}")

if num_duplicates > 0:
    print(weather_data[duplicate_rows])

Number of duplicate rows: 1
                    date   ff       t  ht_neige  rr1  ww     n  etat_sol
2018 2020-11-20 18:00:00  1.0  278.15       0.0  0.0   1  50.0       0.0


In [7]:
weather_data = weather_data.drop_duplicates(subset=["date"], keep="first")

In [8]:
column_renames = {
        "ff": "wind_speed",
        "t": "temperature",
        "ww": "weather_condition",
        "n": "cloud_cover",
        "etat_sol": "ground_state",
        "ht_neige": "snow_height",
        "rr1": "precipitation_last_hour",
        "date": "date"
    }
weather_data = weather_data.rename(columns=column_renames)

In [9]:
weather_data

,date,wind_speed,temperature,snow_height,precipitation_last_hour,weather_condition,cloud_cover,ground_state
0,2021-01-01 00:00:00,1.8,272.75,0.00,0.0,2,10.0,1.0
1,2021-01-01 03:00:00,1.7,271.25,0.00,0.0,40,25.0,1.0
2,2021-01-01 06:00:00,2.6,271.95,0.00,0.0,3,90.0,1.0
3,2021-01-01 09:00:00,1.7,272.45,0.01,0.0,10,50.0,13.0
4,2021-01-01 12:00:00,1.0,276.95,-0.01,0.0,2,90.0,11.0
...,...,...,...,...,...,...,...,...
3317,2020-09-30 09:00:00,4.4,289.95,0.00,0.0,3,90.0,0.0
3318,2020-09-30 12:00:00,4.9,292.05,0.00,0.0,1,90.0,0.0
3319,2020-09-30 15:00:00,4.1,291.55,0.00,0.0,1,90.0,0.0
3320,2020-09-30 18:00:00,2.7,290.15,0.00,0.0,3,100.0,0.0


In [10]:
print(weather_data.dtypes)

date                       datetime64[ns]
wind_speed                        float64
temperature                       float64
snow_height                       float64
precipitation_last_hour           float64
weather_condition                   int64
cloud_cover                       float64
ground_state                      float64
dtype: object


In [11]:
print(weather_data["weather_condition"].unique())

[ 2 40  3 10  1  0 50 60 20 21 26 61 58 51 73 66 62 25 80 63 11 71 22 43
 29 15 17 81 44 82 95 13 16 41 47 28 46 45 42 68 92 27]


In [12]:
print(data.dtypes)

counter_id                         category
counter_name                       category
site_id                               int64
site_name                          category
bike_count                          float64
date                         datetime64[us]
counter_installation_date    datetime64[us]
coordinates                        category
counter_technical_id               category
latitude                            float64
longitude                           float64
log_bike_count                      float64
dtype: object


In [13]:
weather_data["date"] = weather_data["date"].astype("datetime64[us]")

In [14]:
# Returns True if any column has missing values
weather_data.isna().any()

date                       False
wind_speed                 False
temperature                False
snow_height                 True
precipitation_last_hour     True
weather_condition          False
cloud_cover                 True
ground_state                True
dtype: bool

In [15]:
# weather data is only recorded every three hours so we need to resample
weather_data['date'] = pd.to_datetime(weather_data['date'])
weather_data.set_index('date', inplace=True)

weather_data_resampled = weather_data.resample('h').interpolate(method='linear')

weather_data_resampled = weather_data_resampled.reset_index()

In [16]:
weather_data

,wind_speed,temperature,snow_height,precipitation_last_hour,weather_condition,cloud_cover,ground_state
date,,,,,,,
2021-01-01 00:00:00,1.8,272.75,0.00,0.0,2,10.0,1.0
2021-01-01 03:00:00,1.7,271.25,0.00,0.0,40,25.0,1.0
2021-01-01 06:00:00,2.6,271.95,0.00,0.0,3,90.0,1.0
2021-01-01 09:00:00,1.7,272.45,0.01,0.0,10,50.0,13.0
2021-01-01 12:00:00,1.0,276.95,-0.01,0.0,2,90.0,11.0
...,...,...,...,...,...,...,...
2020-09-30 09:00:00,4.4,289.95,0.00,0.0,3,90.0,0.0
2020-09-30 12:00:00,4.9,292.05,0.00,0.0,1,90.0,0.0
2020-09-30 15:00:00,4.1,291.55,0.00,0.0,1,90.0,0.0


In [17]:
merged_data = pd.merge(data, weather_data_resampled, how='inner', on='date')

In [18]:
print(type(merged_data))

<class 'pandas.core.frame.DataFrame'>


In [19]:
merged_data['date'] = pd.to_datetime(merged_data['date'])

In [20]:
def _extract_date_components(X):
    X = X.copy()
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour
    return X

In [21]:
merged_data = _extract_date_components(merged_data)

In [22]:
merged_data

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,...,snow_height,precipitation_last_hour,weather_condition,cloud_cover,ground_state,year,month,day,weekday,hour
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0.0,0.0,1.666667,0.000000,0.0,2020,9,1,1,2
1,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0.0,0.0,2.000000,0.000000,0.0,2020,9,1,1,3
2,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0.0,0.0,2.333333,3.333333,0.0,2020,9,1,1,4
3,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0.0,0.0,3.000000,60.000000,0.0,2020,9,1,1,15
4,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0.0,0.0,2.000000,90.000000,0.0,2020,9,1,1,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496822,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,445.0,2021-09-09 06:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,0.0,0.0,10.000000,100.000000,1.0,2021,9,9,3,6
496823,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,145.0,2021-09-09 10:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,0.0,0.0,1.333333,91.666667,0.0,2021,9,9,3,10
496824,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,218.0,2021-09-09 15:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,0.0,0.0,2.000000,75.000000,0.0,2021,9,9,3,15
496825,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,21.0,2021-09-09 22:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,0.0,0.0,2.000000,66.666667,1.0,2021,9,9,3,22


# Feature Engineering

In [23]:
print(merged_data.columns)

Index(['counter_id', 'counter_name', 'site_id', 'site_name', 'bike_count',
       'date', 'counter_installation_date', 'coordinates',
       'counter_technical_id', 'latitude', 'longitude', 'log_bike_count',
       'wind_speed', 'temperature', 'snow_height', 'precipitation_last_hour',
       'weather_condition', 'cloud_cover', 'ground_state', 'year', 'month',
       'day', 'weekday', 'hour'],
      dtype='object')


In [24]:
def remove_outliers(merged_data):
    merged_data["date_truncated"] = merged_data["date"].dt.floor("D")

    cleaned_data = (
        merged_data.groupby(["counter_name", "date_truncated"])
        ["log_bike_count"].sum()
        .to_frame()
        .reset_index()
        .query("log_bike_count == 0")
        [["counter_name", "date_truncated"]]
        .merge(merged_data, on=["counter_name", "date_truncated"], how="right", indicator=True)
        .query("_merge == 'right_only'")
        .drop(columns=["_merge", "date_truncated"])
    )

    return cleaned_data

In [25]:
merged_data = remove_outliers(merged_data)

/var/folders/5r/98s4tgxs2dvck5m2r8s2mh_m0000gn/T/ipykernel_51597/336265643.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_data.groupby(["counter_name", "date_truncated"])


In [26]:
merged_data

,counter_name,counter_id,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,...,snow_height,precipitation_last_hour,weather_condition,cloud_cover,ground_state,year,month,day,weekday,hour
0,28 boulevard Diderot E-O,100007049-102007049,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0.0,0.0,1.666667,0.000000,0.0,2020,9,1,1,2
1,28 boulevard Diderot E-O,100007049-102007049,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0.0,0.0,2.000000,0.000000,0.0,2020,9,1,1,3
2,28 boulevard Diderot E-O,100007049-102007049,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0.0,0.0,2.333333,3.333333,0.0,2020,9,1,1,4
3,28 boulevard Diderot E-O,100007049-102007049,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0.0,0.0,3.000000,60.000000,0.0,2020,9,1,1,15
4,28 boulevard Diderot E-O,100007049-102007049,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0.0,0.0,2.000000,90.000000,0.0,2020,9,1,1,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496822,254 rue de Vaugirard SO-NE,300014702-353245971,300014702,254 rue de Vaugirard,445.0,2021-09-09 06:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,0.0,0.0,10.000000,100.000000,1.0,2021,9,9,3,6
496823,254 rue de Vaugirard SO-NE,300014702-353245971,300014702,254 rue de Vaugirard,145.0,2021-09-09 10:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,0.0,0.0,1.333333,91.666667,0.0,2021,9,9,3,10
496824,254 rue de Vaugirard SO-NE,300014702-353245971,300014702,254 rue de Vaugirard,218.0,2021-09-09 15:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,0.0,0.0,2.000000,75.000000,0.0,2021,9,9,3,15
496825,254 rue de Vaugirard SO-NE,300014702-353245971,300014702,254 rue de Vaugirard,21.0,2021-09-09 22:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,0.0,0.0,2.000000,66.666667,1.0,2021,9,9,3,22


In [27]:
merged_data.columns

Index(['counter_name', 'counter_id', 'site_id', 'site_name', 'bike_count',
       'date', 'counter_installation_date', 'coordinates',
       'counter_technical_id', 'latitude', 'longitude', 'log_bike_count',
       'wind_speed', 'temperature', 'snow_height', 'precipitation_last_hour',
       'weather_condition', 'cloud_cover', 'ground_state', 'year', 'month',
       'day', 'weekday', 'hour'],
      dtype='object')

In [28]:
def _add_rush_hour_indicator(X):
    X = X.copy()
    X["is_rush_hour"] = X["hour"].isin([7, 8, 9, 17, 18, 19]).astype(int)
    return X

In [29]:
def _add_weekend_indicator(X):
    X = X.copy()
    X["is_weekend"] = (X["weekday"] >= 5).astype(int)
    return X

In [30]:
def _add_holiday_indicator(X):
    X = X.copy()
    france_holidays = holidays.FR()  # Use the holidays package for France
    X["is_holiday"] = X["date"].dt.date.apply(lambda x: 1 if x in france_holidays else 0)
    return X

In [31]:
tourist_locations = {
    "Eiffel Tower": (48.8584, 2.2945),
    "Louvre Museum": (48.8606, 2.3376),
    "Notre Dame Cathedral": (48.852968, 2.349902),
    "Sacré-Cœur": (48.8867, 2.3431),
    "Arc de Triomphe": (48.8738, 2.2950),
    "Châtelet Station": (48.8581, 2.3470),
    "Gare du Nord": (48.8808, 2.3553),
    "Montparnasse Tower": (48.8422, 2.3211),
    "Place de la République": (48.8673, 2.3632),
    "Place de la Bastille": (48.853, 2.369),
    "Opéra Garnier": (48.8719, 2.3316),
}

In [32]:
def _haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Add minimum tourist distance feature
def _add_tourist_proximity(X):
    X = X.copy()
    distances = []

    # Calculate distances for each tourist location
    for location, (lat_tourist, lon_tourist) in tourist_locations.items():
        distance = _haversine(X["latitude"], X["longitude"], lat_tourist, lon_tourist)
        distances.append(distance)

    # Add minimum distance as a new column
    X["min_tourist_distance"] = np.min(distances, axis=0)
    return X

In [33]:
merged_data = _add_rush_hour_indicator(merged_data)
merged_data = _add_weekend_indicator(merged_data)
merged_data = _add_holiday_indicator(merged_data)
merged_data = _add_tourist_proximity(merged_data)

In [34]:
merged_data

,counter_name,counter_id,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,...,ground_state,year,month,day,weekday,hour,is_rush_hour,is_weekend,is_holiday,min_tourist_distance
0,28 boulevard Diderot E-O,100007049-102007049,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0.0,2020,9,1,1,2,0,0,0,0.906809
1,28 boulevard Diderot E-O,100007049-102007049,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0.0,2020,9,1,1,3,0,0,0,0.906809
2,28 boulevard Diderot E-O,100007049-102007049,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0.0,2020,9,1,1,4,0,0,0,0.906809
3,28 boulevard Diderot E-O,100007049-102007049,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0.0,2020,9,1,1,15,0,0,0,0.906809
4,28 boulevard Diderot E-O,100007049-102007049,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0.0,2020,9,1,1,18,1,0,0,0.906809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496822,254 rue de Vaugirard SO-NE,300014702-353245971,300014702,254 rue de Vaugirard,445.0,2021-09-09 06:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,1.0,2021,9,9,3,6,0,0,0,1.425110
496823,254 rue de Vaugirard SO-NE,300014702-353245971,300014702,254 rue de Vaugirard,145.0,2021-09-09 10:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,0.0,2021,9,9,3,10,0,0,0,1.425110
496824,254 rue de Vaugirard SO-NE,300014702-353245971,300014702,254 rue de Vaugirard,218.0,2021-09-09 15:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,0.0,2021,9,9,3,15,0,0,0,1.425110
496825,254 rue de Vaugirard SO-NE,300014702-353245971,300014702,254 rue de Vaugirard,21.0,2021-09-09 22:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,1.0,2021,9,9,3,22,0,0,0,1.425110


In [35]:
# Define feature dropping for low predictive value and date
def _drop_low_value_features(X):
    X = X.copy()
    X.drop(columns=['site_id', 'counter_id', 'counter_installation_date', 
                    'counter_technical_id', 'coordinates', 'latitude', 'longitude'], inplace=True)
    return X

In [36]:
merged_data = _drop_low_value_features(merged_data)

In [37]:
merged_data.columns

Index(['counter_name', 'site_name', 'bike_count', 'date', 'log_bike_count',
       'wind_speed', 'temperature', 'snow_height', 'precipitation_last_hour',
       'weather_condition', 'cloud_cover', 'ground_state', 'year', 'month',
       'day', 'weekday', 'hour', 'is_rush_hour', 'is_weekend', 'is_holiday',
       'min_tourist_distance'],
      dtype='object')

# Encoding

In [38]:

def _apply_cyclical_encoding(X):
    X = X.copy()
    # Apply cyclical encoding
    X["hour_sin"] = np.sin(2 * np.pi * X["hour"] / 24)
    X["hour_cos"] = np.cos(2 * np.pi * X["hour"] / 24)
    X["month_sin"] = np.sin(2 * np.pi * X["month"] / 12)
    X["month_cos"] = np.cos(2 * np.pi * X["month"] / 12)
    X["weekday_sin"] = np.sin(2 * np.pi * X["weekday"] / 7)
    X["weekday_cos"] = np.cos(2 * np.pi * X["weekday"] / 7)
    return X

## MODELS

In [39]:
X = merged_data.drop(columns=["bike_count", "log_bike_count"])
y = merged_data["log_bike_count"]

#### Basic Linear Model

In [40]:
# from sklearn.model_selection import train_test_split

# # Temporal train-test split
# def train_test_split_temporal(X, y, delta_threshold="30 days"):
#     cutoff_date = X["date"].max() - pd.Timedelta(delta_threshold)
#     mask = (X["date"] <= cutoff_date)
#     X_train, X_valid = X.loc[mask], X.loc[~mask]
#     y_train, y_valid = y[mask], y[~mask]
#     return X_train, y_train, X_valid, y_valid

# # Split the data
# X_train, y_train, X_valid, y_valid = train_test_split_temporal(X, y)

# print(f"Train set: {X_train.shape}, Validation set: {X_valid.shape}")

In [41]:
from sklearn.model_selection import train_test_split

# Perform a standard train-test split (randomized split)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, 
    y, 
    test_size=0.2,  # 20% of data for validation
    random_state=42  # Ensures reproducibility
)

print(f"Train set: {X_train.shape}, Validation set: {X_valid.shape}")

Train set: (393295, 19), Validation set: (98324, 19)


In [42]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

# Define feature groups
numerical_features = [
    "wind_speed", "temperature", "snow_height", "precipitation_last_hour", # "weather_condition", "cloud_cover", "ground_state"
]
categorical_features = ["counter_name", "site_name"]
new_features = ["is_rush_hour", "is_weekend", "is_holiday", "min_tourist_distance"]
cyclical_features = ["hour_sin", "hour_cos", "month_sin", "month_cos", "weekday_sin", "weekday_cos"]

# Preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),  # Handle NaN values
    # ("scaler", StandardScaler())  # Standard scaling
])

# Preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
     ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
 ])

# Preprocessing for cyclical encoding
cyclical_transformer = Pipeline(steps=[
    ("cyclical_encoding", FunctionTransformer(_apply_cyclical_encoding, validate=False)),
    #("scaler", StandardScaler())  # Standard scaling for cyclical features
])

# Combine all preprocessors into a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
        ("cyclical", cyclical_transformer, ["hour", "month", "weekday"]),
        ("new_features", "passthrough", new_features),  # Pass these features as is
        ("time_features", "passthrough", ["weekday", "hour"])  # Pass weekday and hour directly
    ]
)

# Define the full pipeline with preprocessing and the model
linear_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("linear_model", LinearRegression())
])

# Fit the pipeline to the training data
linear_pipeline.fit(X_train, y_train)

# Make predictions
y_train_pred = linear_pipeline.predict(X_train)
y_valid_pred = linear_pipeline.predict(X_valid)

# Evaluate the model
from sklearn.metrics import mean_squared_error

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
valid_rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))

print(f"Train RMSE: {train_rmse:.2f}")
print(f"Validation RMSE: {valid_rmse:.2f}")

Train RMSE: 0.83
Validation RMSE: 0.83


In [43]:
linear_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['wind_speed', 'temperature',
                                                   'snow_height',
                                                   'precipitation_last_hour']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['counter_name',
                                                   'site_name']),
                                                 ('cyclical',
                                                  Pipeline(steps=[('cyclical_encoding',
                                                                   FunctionTransformer(func=<function _apply_cyclical_encoding at 0x1462e5510>))]),
                                                  ['hour', 'month', 'weekday']),
                                                 ('new_features', 'passthrough',
                                                  ['is_rush_hour', 'is_weekend',
                                                   'is_holiday',
                                                   'min_tourist_distance']),
                                                 ('time_features',
                                                  'passthrough',
                                                  ['weekday', 'hour'])])),
                ('linear_model', LinearRegression())])

#### Ridge Regression

In [44]:
ridge_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("ridge_model", Ridge(alpha=1.0))  # Alpha is the regularization strength
])

# Fit the pipeline to the training data
ridge_pipeline.fit(X_train, y_train)

# Make predictions
y_train_pred = ridge_pipeline.predict(X_train)
y_valid_pred = ridge_pipeline.predict(X_valid)

# Evaluate the model
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
valid_rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))

print(f"Ridge Regression - Train RMSE: {train_rmse:.2f}")
print(f"Ridge Regression - Validation RMSE: {valid_rmse:.2f}")

Ridge Regression - Train RMSE: 0.83
Ridge Regression - Validation RMSE: 0.83


In [45]:
ridge_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['wind_speed', 'temperature',
                                                   'snow_height',
                                                   'precipitation_last_hour']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['counter_name',
                                                   'site_name']),
                                                 ('cyclical',
                                                  Pipeline(steps=[('cyclical_encoding',
                                                                   FunctionTransformer(func=<function _apply_cyclical_encoding at 0x1462e5510>))]),
                                                  ['hour', 'month', 'weekday']),
                                                 ('new_features', 'passthrough',
                                                  ['is_rush_hour', 'is_weekend',
                                                   'is_holiday',
                                                   'min_tourist_distance']),
                                                 ('time_features',
                                                  'passthrough',
                                                  ['weekday', 'hour'])])),
                ('ridge_model', Ridge())])

### Random Forest

In [46]:
# rf_pipeline = Pipeline(steps=[
#     ("preprocessor", preprocessor),
#     ("rf_model", RandomForestRegressor(n_estimators=100, random_state=42))
# ])

# # Fit the pipeline to the training data
# rf_pipeline.fit(X_train, y_train)

# # Make predictions
# y_train_pred = rf_pipeline.predict(X_train)
# y_valid_pred = rf_pipeline.predict(X_valid)

# # Evaluate the model
# train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
# valid_rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))

# print(f"Random Forest - Train RMSE: {train_rmse:.2f}")
# print(f"Random Forest - Validation RMSE: {valid_rmse:.2f}")

long load times, so disables for now, last RMSE was 1.22, so we moved forward to XGBoost

#### HistgradientBoost

In [47]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import numpy as np

preprocessor_hgb = ColumnTransformer(
    transformers=[
        # ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
        ("cyclical", cyclical_transformer, ["hour", "month", "weekday"]),
        ("new_features", "passthrough", new_features),  # Pass these features as is
        ("time_features", "passthrough", ["weekday", "hour"]),  # Pass weekday and hour directly
    ]
)

hgb_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("hgb_model", HistGradientBoostingRegressor(random_state=42))
])

# Fit the pipeline to the training data
hgb_pipeline.fit(X_train, y_train)

# Make predictions
y_train_pred = hgb_pipeline.predict(X_train)
y_valid_pred = hgb_pipeline.predict(X_valid)

# Evaluate the model
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
valid_rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))

print(f"HistGradientBoostingRegressor - Train RMSE: {train_rmse:.2f}")
print(f"HistGradientBoostingRegressor - Validation RMSE: {valid_rmse:.2f}")


HistGradientBoostingRegressor - Train RMSE: 0.49
HistGradientBoostingRegressor - Validation RMSE: 0.49


In [48]:
hgb_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['wind_speed', 'temperature',
                                                   'snow_height',
                                                   'precipitation_last_hour']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['counter_name',
                                                   'site_name']),
                                                 ('cyclical',
                                                  Pipeline(steps=[('cyclical_encoding',
                                                                   FunctionTransformer(func=<function _apply_cyclical_encoding at 0x1462e5510>))]),
                                                  ['hour', 'month', 'weekday']),
                                                 ('new_features', 'passthrough',
                                                  ['is_rush_hour', 'is_weekend',
                                                   'is_holiday',
                                                   'min_tourist_distance']),
                                                 ('time_features',
                                                  'passthrough',
                                                  ['weekday', 'hour'])])),
                ('hgb_model', HistGradientBoostingRegressor(random_state=42))])

#### XGBOOST

We chose to disregard XGBoost. Even though it was a good performing model, it was difficult to resolve compatibility issues with scikit-learn. 

### Cross Validation

In [49]:
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import numpy as np

# Define cross-validation strategy
cv = TimeSeriesSplit(n_splits=6)

# Define a helper function for cross-validation
def cross_validate_model(model, X, y, cv):
    scores = cross_val_score(
        model, X, y, cv=cv, scoring="neg_root_mean_squared_error"
    )
    print(f"RMSE (all folds): {-scores.mean():.3f} ± {(-scores).std():.3f}")
    return scores

# Cross-validate each model
print("Cross-validation for Ridge Regression:")
cross_validate_model(ridge_pipeline, X_train, y_train, cv)

print("Cross-validation for HistGradientBoosting:")
cross_validate_model(hgb_pipeline, X_train, y_train, cv)

# print("Cross-validation for Random Forest:")
# cross_validate_model(random_forest_model, X_train, y_train, cv)

Cross-validation for Ridge Regression:
RMSE (all folds): 0.831 ± 0.001
Cross-validation for HistGradientBoosting:
RMSE (all folds): 0.490 ± 0.001


array([-0.49156299, -0.49169251, -0.48986606, -0.48939188, -0.48846611,
       -0.48850473])

**Cross-Validation Results**
- Ridge Regression: RMSE = 1.075 ± 0.177 (higher error, less consistent).
- HistGradientBoosting: RMSE = 0.898 ± 0.213 (better performance and reasonable consistency).
- Conclusion: HistGradientBoosting outperforms Ridge Regression.

Note: We also chose to disregard RandomForest due to very long load times. 

### Hyperparameter Tuning

In [50]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import numpy as np

# Define parameter distributions
param_distributions = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_iter': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'min_samples_leaf': [20, 30, 50],
    'l2_regularization': [1.0, 5.0, 10.0]
}

# Initialize the HistGradientBoosting model
hgb_model = HistGradientBoostingRegressor(
    random_state=42,
    early_stopping=True,  # Enable early stopping
    validation_fraction=0.1,  # Use 10% of training data for early stopping
    n_iter_no_change=10  # Stop if no improvement after 10 iterations
)

# Wrap the model in a pipeline if preprocessing is required
hgb_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),  # Assuming a preprocessor is defined
    ("regressor", hgb_model)
])

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=hgb_pipeline,
    param_distributions={'regressor__' + k: v for k, v in param_distributions.items()},
    n_iter=20,  # Number of parameter combinations to try
    scoring='neg_root_mean_squared_error',  # RMSE as scoring metric
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Print updates during the search
    random_state=42,
    n_jobs=-1  # Use all available CPUs
)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Use the best pipeline from RandomizedSearchCV
best_hgb_pipeline = random_search.best_estimator_

# Make predictions
y_train_pred = best_hgb_pipeline.predict(X_train)
y_valid_pred = best_hgb_pipeline.predict(X_valid)

# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
valid_rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))

# Print results
print(f"Best Parameters: {random_search.best_params_}")
print(f"Best RMSE (CV): {abs(random_search.best_score_):.2f}")
print(f"Train RMSE: {train_rmse:.2f}")
print(f"Validation RMSE: {valid_rmse:.2f}")

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best Parameters: {'regressor__min_samples_leaf': 50, 'regressor__max_iter': 300, 'regressor__max_depth': 5, 'regressor__learning_rate': 0.1, 'regressor__l2_regularization': 5.0}
Best RMSE (CV): 0.47
Train RMSE: 0.46
Validation RMSE: 0.47


In [51]:
from sklearn.model_selection import cross_val_score
best_pipeline = random_search.best_estimator_

cv_scores = cross_val_score(
    best_pipeline, X_train, y_train, scoring="neg_root_mean_squared_error", cv=3
)
print(f"Cross-validated RMSE: {abs(cv_scores.mean()):.3f} ± {cv_scores.std():.3f}")

Cross-validated RMSE: 0.469 ± 0.002


**Check if preprocessing was applied consistently**

In [52]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['wind_speed', 'temperature', 'snow_height',
                                  'precipitation_last_hour']),
                                ('cat',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['counter_name', 'site_name']),
                                ('cyclical',
                                 Pipeline(steps=[('cyclical_encoding',
                                                  FunctionTransformer(func=<function _apply_cyclical_encoding at 0x1462e5510>))]),
                                 ['hour', 'month', 'weekday']),
                                ('new_features', 'passthrough',
                                 ['is_rush_hour', 'is_weekend', 'is_holiday',
                                  'min_tourist_distance']),
                                ('time_features', 'passthrough',
                                 ['weekday', 'hour'])])

In [53]:
X_valid_transformed = preprocessor.transform(X_valid)
print(X_valid_transformed[:5])  # Check transformed values

[[ 4.43333333e+00  2.88516667e+02  0.00000000e+00 -1.00000000e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000

In [54]:
X_train_transformed = preprocessor.transform(X_train)
print(X_train_transformed[:5])  # Check training transformations

[[ 6.63333333e+00  2.85316667e+02  0.00000000e+00 -3.33333333e-02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000

In [55]:
print(pd.DataFrame(X_train_transformed).describe())
print(pd.DataFrame(X_valid_transformed).describe())

                 0              1              2              3    \
count  393295.000000  393295.000000  393295.000000  393295.000000   
mean        3.727270     285.609547      -0.000102       0.076221   
std         1.912538       6.979711       0.002418       0.443570   
min         0.000000     267.650000      -0.020000      -0.100000   
25%         2.300000     280.550000       0.000000       0.000000   
50%         3.500000     285.283333       0.000000       0.000000   
75%         4.966667     290.583333       0.000000       0.000000   
max        12.700000     307.450000       0.040000      14.900000   

                 4              5              6              7    \
count  393295.000000  393295.000000  393295.000000  393295.000000   
mean        0.016781       0.016855       0.018274       0.018116   
std         0.128451       0.128728       0.133940       0.133372   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.0000

In [56]:
print(X_valid_transformed[:5])

[[ 4.43333333e+00  2.88516667e+02  0.00000000e+00 -1.00000000e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000

In [57]:
print(f"Reported Best RMSE (CV): {abs(random_search.best_score_):.3f}")
print(f"Cross-validated RMSE (manual): {abs(cv_scores.mean()):.3f}")

Reported Best RMSE (CV): 0.469
Cross-validated RMSE (manual): 0.469


# Test data

In [58]:
test_data = pd.read_parquet(Path("data") / "final_test.parquet")

In [59]:
test_data.head()

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 01:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
1,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 13:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
2,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 17:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
3,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 19:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
4,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 22:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429


In [60]:
# Merge external data (weather data) with the test dataset
test_data_merged = pd.merge(test_data, weather_data_resampled, how='inner', on='date')

# Add the manually created features
test_data_merged = _extract_date_components(test_data_merged)
test_data_merged = _add_rush_hour_indicator(test_data_merged)
test_data_merged = _add_weekend_indicator(test_data_merged)
test_data_merged = _add_holiday_indicator(test_data_merged)
test_data_merged = _add_tourist_proximity(test_data_merged)
test_data_merged = _drop_low_value_features(test_data_merged)

# Ensure columns match the model input (drop unnecessary columns, align with training set)
test_data_merged = test_data_merged[X_train.columns]

In [61]:
test_data_merged

,counter_name,site_name,date,wind_speed,temperature,snow_height,precipitation_last_hour,weather_condition,cloud_cover,ground_state,year,month,day,weekday,hour,is_rush_hour,is_weekend,is_holiday,min_tourist_distance
0,28 boulevard Diderot E-O,28 boulevard Diderot,2021-09-10 01:00:00,2.500000,291.483333,0.0,0.000000,1.000000,96.666667,0.666667,2021,9,10,4,1,0,0,0,0.906809
1,28 boulevard Diderot E-O,28 boulevard Diderot,2021-09-10 13:00:00,2.900000,294.816667,0.0,1.066667,17.000000,70.000000,0.000000,2021,9,10,4,13,0,0,0,0.906809
2,28 boulevard Diderot E-O,28 boulevard Diderot,2021-09-10 17:00:00,3.500000,294.550000,0.0,0.533333,54.333333,70.000000,0.666667,2021,9,10,4,17,1,0,0,0.906809
3,28 boulevard Diderot E-O,28 boulevard Diderot,2021-09-10 19:00:00,2.400000,292.850000,0.0,0.533333,54.333333,75.000000,1.000000,2021,9,10,4,19,1,0,0,0.906809
4,28 boulevard Diderot E-O,28 boulevard Diderot,2021-09-10 22:00:00,2.000000,290.716667,0.0,0.000000,1.666667,70.000000,0.666667,2021,9,10,4,22,0,0,0,0.906809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51435,254 rue de Vaugirard SO-NE,254 rue de Vaugirard,2021-10-18 11:00:00,3.400000,288.083333,0.0,0.000000,2.333333,90.000000,0.000000,2021,10,18,0,11,0,0,0,1.425110
51436,254 rue de Vaugirard SO-NE,254 rue de Vaugirard,2021-10-18 15:00:00,5.200000,291.550000,0.0,0.000000,3.000000,90.000000,0.000000,2021,10,18,0,15,0,0,0,1.425110
51437,254 rue de Vaugirard SO-NE,254 rue de Vaugirard,2021-10-18 17:00:00,3.000000,290.550000,0.0,-0.066667,41.666667,96.666667,0.000000,2021,10,18,0,17,1,0,0,1.425110
51438,254 rue de Vaugirard SO-NE,254 rue de Vaugirard,2021-10-18 18:00:00,1.900000,290.050000,0.0,-0.100000,61.000000,100.000000,0.000000,2021,10,18,0,18,1,0,0,1.425110


In [62]:
test_predictions = best_hgb_pipeline.predict(test_data_merged)

In [63]:
submission_df = pd.DataFrame({
    "Id": range(len(test_predictions)),  # Assuming sequential indices for Id
    "log_bike_count": test_predictions
})


In [64]:
# Save to CSV
submission_file_path = "submission.csv"
submission_df.to_csv(submission_file_path, index=False)

print(f"Submission file saved to {submission_file_path}")

Submission file saved to submission.csv
